In [36]:
import numpy as np
import pandas as pd
import re
import re as reg
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
data = pd.read_csv("Kasus1.csv", delimiter= ';')
data

,screen_name,username,user_id,tweet_id,tweet_url,timestamp,timestamp_epochs,text,mengandung pelecehan,text_html,...,has_media,img_urls,video_url,likes,retweets,replies,is_replied,is_reply_to,parent_tweet_id,reply_to_users
0,BimaBurhanuddin,Burhanuddin Bima,3058961791,"1,12E+18",/BimaBurhanuddin/status/1124100611925000193,02/05/2019 23:57,1556841445,Apa yg bisa diharapkan dari politisi gaya pela...,1,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,0,0,0,False,False,NaN,[]
1,ngowatchindo,Waw Lam'alif,"8,95E+17","1,12E+18",/ngowatchindo/status/1124099972046163968,02/05/2019 23:54,1556841292,"Kalau sy jadi ⁦@DennyJA_WORLD⁩ , Syahganda aka...",1,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,1,1,0,False,False,NaN,[]
2,socmed_anak,AnakSocmed,174945628,"1,12E+18",/socmed_anak/status/1124098250083131393,02/05/2019 23:48,1556840882,Di @PDemokrat banyak org2 yg punya akal sehat ...,1,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,0,0,0,False,False,NaN,[]
3,PelajaranOnline,PelajaranOnline,589466301,"1,12E+18",/PelajaranOnline/status/1124087714154975232,02/05/2019 23:06,1556838370,Cypridophobia adalah fobia atau takut pada pel...,0,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,0,0,0,False,False,NaN,[]
4,Lee_Botak,Lee Botak,509046818,"1,12E+18",/Lee_Botak/status/1124084310640869379,02/05/2019 22:52,1556837558,Yang RAMAI kerja waktu malam ni biasanya PELAC...,1,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,0,0,0,False,False,NaN,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,OposisiRasional,Oposisi_Rasional,"1,13E+18","1,17E+18",/OposisiRasional/status/1174274919196188672,18/09/2019 10:52,1568803932,Letakan DiKepalak Sepatunya Klo Tidak Mau Koto...,0,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,56,6,1,True,True,"1,17E+18","[{'screen_name': 'LisaAmartatara3', 'user_id':..."
396,Adnity05,Adnity,"1,11E+18","1,17E+18",/Adnity05/status/1174270298792898560,18/09/2019 10:33,1568802830,Media pelacur lahir dari rahim yg tak jujur. h...,1,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,0,0,0,False,False,NaN,[]
397,adebaihaki1,Bakayarou,"1,09E+18","1,22E+18",/adebaihaki1/status/1215058126501580802,08/01/2020 23:50,1578527406,Jadi pelacur ko nanti dijawa nak. Jangan...,0,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,0,0,1,True,True,"1,21E+18","[{'screen_name': 'mhdaziz_', 'user_id': '99137..."
398,RICHARDSAMBORA3,RICHARD SAMBORA,"1,16E+18","1,17E+18",/RICHARDSAMBORA3/status/1174272471991799808,18/09/2019 10:42,1568803348,Kok jadi pelacur,1,"<p class=""TweetTextSize js-tweet-text tweet-te...",...,False,[],NaN,0,0,0,False,True,"1,17E+18","[{'screen_name': 'Openbo15989801', 'user_id': ..."


In [11]:
data = data.drop(['screen_name', 'username', 'user_id', 'tweet_id', 'tweet_url', 'timestamp', 'timestamp_epochs', 'mengandung pelecehan',
                 'text_html', 'links', 'hashtags', 'has_media', 'video_url', 'likes', 'retweets', 'replies', 'is_replied', 'is_reply_to',
                 'parent_tweet_id', 'reply_to_users', 'img_urls'], axis=1)

In [12]:
data

,text
0,Apa yg bisa diharapkan dari politisi gaya pela...
1,"Kalau sy jadi ⁦@DennyJA_WORLD⁩ , Syahganda aka..."
2,Di @PDemokrat banyak org2 yg punya akal sehat ...
3,Cypridophobia adalah fobia atau takut pada pel...
4,Yang RAMAI kerja waktu malam ni biasanya PELAC...
...,...
395,Letakan DiKepalak Sepatunya Klo Tidak Mau Koto...
396,Media pelacur lahir dari rahim yg tak jujur. h...
397,Jadi pelacur ko nanti dijawa nak. Jangan...
398,Kok jadi pelacur


In [13]:
#Preprocessing
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

slangs={'yg':'yang', 'tdk':'tidak', 'pd':'pada', 'mlh':'malah', 'jgn':'jangan', 'jg':'juga', 'tp':'tapi', 'blkg': 'belakang', 'dr':'dari', 'klo':'kalo', 'lg':'lagi', 'btw':'buat',
        'dlm':'dalam','dgn':'dengan', 'poto':'foto', 'g':'tidak', 'n':'dan', 'ad':'ada', 'brp': 'berapa', "abis": "habis", "ad": "ada", "adlh": "adalah", "afaik": "as far as i know", 
        "ahaha": "haha", "aj": "saja", "ajep-ajep": "dunia gemerlap", "ak": "saya", "akika": "aku", "akkoh": "aku", "akuwh": "aku", "alay": "norak", "alow": "halo", "ambilin": "ambilkan", 
        "ancur": "hancur", "anjrit": "anjing", "anter": "antar", "ap2": "apa-apa", "apasih": "apa sih", "apes": "sial", "aps": "apa", "aq": "saya", "aquwh": "aku", "asbun": "asal bunyi", 
        "aseekk": "asyik", "asekk": "asyik", "asem": "asam", "aspal": "asli tetapi palsu", "astul": "asal tulis", "ato": "atau", "au ah": "tidak mau tahu", "awak": "saya", "ay": "sayang", 
        "ayank": "sayang", "b4": "sebelum", "bakalan": "akan", "bandes": "bantuan desa", "bangedh": "banget", "banpol": "bantuan polisi", "banpur": "bantuan tempur", "basbang": "basi", 
        "bcanda": "bercanda", "bdg": "bandung", "begajulan": "nakal", "beliin": "belikan", "bencong": "banci", "bentar": "sebentar", "ber3": "bertiga", "beresin": "membereskan", "bete": 
        "bosan", "beud": "banget", "bg": "abang", "bgmn": "bagaimana", "bgt": "banget", "bijimane": "bagaimana", "bintal": "bimbingan mental", "bkl": "akan", "bknnya": "bukannya", 
        "blegug": "bodoh", "blh": "boleh", "bln": "bulan", "blum": "belum", "bnci": "benci", "bnran": "yang benar", "bodor": "lucu", "bokap": "ayah", "boker": "buang air besar", "bokis": 
        "bohong", "boljug": "boleh juga", "bonek": "bocah nekat", "boyeh": "boleh", "br": "baru", "brg": "bareng", "bro": "saudara laki-laki", "bru": "baru", "bs": "bisa", "bsen": "bosan", 
        "bt": "buat", "btw": "ngomong-ngomong", "buaya": "tidak setia", "bubbu": "tidur", "bubu": "tidur", "bumil": "ibu hamil", "bw": "bawa", "bwt": "buat", "byk": "banyak", "byrin": "bayarkan", 
        "cabal": "sabar", "cadas": "keren", "calo": "makelar", "can": "belum", "capcus": "pergi", "caper": "cari perhatian", "ce": "cewek", "cekal": "cegah tangkal", "cemen": "penakut", 
        "cengengesan": "tertawa", "cepet": "cepat", "cew": "cewek", "chuyunk": "sayang", "cimeng": "ganja", "cipika cipiki": "cium pipi kanan cium pipi kiri", "ciyh": "sih", "ckepp": "cakep", 
        "ckp": "cakep", "cmiiw": "correct me if i'm wrong", "cmpur": "campur", "cong": "banci", "conlok": "cinta lokasi", "cowwyy": "maaf", "cp": "siapa", "cpe": "capek", "cppe": "capek", 
        "cucok": "cocok", "cuex": "cuek", "cumi": "Cuma miscall", "cups": "culun", "curanmor": "pencurian kendaraan bermotor", "curcol": "curahan hati colongan", "cwek": "cewek", "cyin": "cinta", 
        "d": "di", "dah": "deh", "dapet": "dapat", "de": "adik", "dek": "adik", "demen": "suka", "deyh": "deh", "dgn": "dengan", "diancurin": "dihancurkan", "dimaafin": "dimaafkan", 
        "dimintak": "diminta", "disono": "di sana", "dket": "dekat", "dkk": "dan kawan-kawan", "dll": "dan lain-lain", "dlu": "dulu", "dngn": "dengan", "dodol": "bodoh", "doku": "uang", 
        "dongs": "dong", "dpt": "dapat", "dri": "dari", "drmn": "darimana", "drtd": "dari tadi", "dst": "dan seterusnya", "dtg": "datang", "duh": "aduh", "duren": "durian", "ed": "edisi", 
        "egp": "emang gue pikirin", "eke": "aku", "elu": "kamu", "emangnya": "memangnya", "emng": "memang", "endak": "tidak", "enggak": "tidak", "envy": "iri", "ex": "mantan", "fax": "facsimile", 
        "fifo": "first in first out", "folbek": "follow back", "fyi": "sebagai informasi", "gaada": "tidak ada uang", "gag": "tidak", "gaje": "tidak jelas", "gak papa": "tidak apa-apa", 
        "gan": "juragan", "gaptek": "gagap teknologi", "gatek": "gagap teknologi", "gawe": "kerja", "gbs": "tidak bisa", "gebetan": "orang yang disuka", "geje": "tidak jelas", 
        "gepeng": "gelandangan dan pengemis", "ghiy": "lagi", "gile": "gila", "gimana": "bagaimana", "gino": "gigi nongol", "githu": "gitu", "gj": "tidak jelas", "gmana": "bagaimana", 
        "gn": "begini", "goblok": "bodoh", "golput": "golongan putih", "gowes": "mengayuh sepeda", "gpny": "tidak punya", "gr": "gede rasa", "gretongan": "gratisan", "gtau": "tidak tahu", 
        "gua": "saya", "guoblok": "goblok", "gw": "saya", "ha": "tertawa", "haha": "tertawa", "hallow": "halo", "hankam": "pertahanan dan keamanan", "hehe": "he", "helo": "halo", "hey": "hai", 
        "hlm": "halaman", "hny": "hanya", "hoax": "isu bohong", "hr": "hari", "hrus": "harus", "hubdar": "perhubungan darat", "huff": "mengeluh", "hum": "rumah", "humz": "rumah", "ilang": "hilang", 
        "ilfil": "tidak suka", "imho": "in my humble opinion", "imoetz": "imut", "item": "hitam", "itungan": "hitungan", "iye": "iya", "ja": "saja", "jadiin": "jadi", "jaim": "jaga image", 
        "jayus": "tidak lucu", "jdi": "jadi", "jem": "jam", "jga": "juga", "jgnkan": "jangankan", "jir": "anjing", "jln": "jalan", "jomblo": "tidak punya pacar", "jubir": "juru bicara", 
        "jutek": "galak", "k": "ke", "kab": "kabupaten", "kabor": "kabur", "kacrut": "kacau", "kadiv": "kepala divisi", "kagak": "tidak", "kalo": "kalau", "kampret": "sialan", 
        "kamtibmas": "keamanan dan ketertiban masyarakat", "kamuwh": "kamu", "kanwil": "kantor wilayah", "karna": "karena", "kasubbag": "kepala subbagian", "katrok": "kampungan", 
        "kayanya": "kayaknya", "kbr": "kabar", "kdu": "harus", "kec": "kecamatan", "kejurnas": "kejuaraan nasional", "kekeuh": "keras kepala", "kel": "kelurahan", "kemaren": "kemarin", 
        "kepengen": "mau", "kepingin": "mau", "kepsek": "kepala sekolah", "kesbang": "kesatuan bangsa", "kesra": "kesejahteraan rakyat", "ketrima": "diterima", "kgiatan": "kegiatan", 
        "kibul": "bohong", "kimpoi": "kawin", "kl": "kalau", "klianz": "kalian", "kloter": "kelompok terbang", "klw": "kalau", "km": "kamu", "kmps": "kampus", "kmrn": "kemarin", "knal": "kenal", 
        "knp": "kenapa", "kodya": "kota madya", "komdis": "komisi disiplin", "komsov": "komunis sovyet", "kongkow": "kumpul bareng teman-teman", "kopdar": "kopi darat", "korup": "korupsi", 
        "kpn": "kapan", "krenz": "keren", "krm": "kirim", "kt": "kita", "ktmu": "ketemu", "ktr": "kantor", "kuper": "kurang pergaulan", "kw": "imitasi", "kyk": "seperti", "la": "lah", 
        "lam": "salam", "lamp": "lampiran", "lanud": "landasan udara", "latgab": "latihan gabungan", "lebay": "berlebihan", "leh": "boleh", "lelet": "lambat", "lemot": "lambat", "lgi": "lagi", 
        "lgsg": "langsung", "liat": "lihat", "litbang": "penelitian dan pengembangan", "lmyn": "lumayan", "lo": "kamu", "loe": "kamu", "lola": "lambat berfikir", "louph": "cinta", "low": "kalau", 
        "lp": "lupa", "luber": "langsung, umum, bebas, dan rahasia", "luchuw": "lucu", "lum": "belum", "luthu": "lucu", "lwn": "lawan", "maacih": "terima kasih", "mabal": "bolos", "macem": "macam", "macih": "masih", "maem": "makan", "magabut": "makan gaji buta", "maho": "homo", "mak jang": "kaget", "maksain": "memaksa", "malem": "malam", "mam": "makan", "maneh": "kamu", "maniez": "manis", "mao": "mau", "masukin": "masukkan", "melu": "ikut", "mepet": "dekat sekali", "mgu": "minggu", "migas": "minyak dan gas bumi", "mikol": "minuman beralkohol", "miras": "minuman keras", "mlah": "malah", "mngkn": "mungkin", "mo": "mau", "mokad": "mati", "moso": "masa", "mpe": "sampai", "msk": "masuk", "mslh": "masalah", "mt": "makan teman", "mubes": "musyawarah besar", "mulu": "melulu", "mumpung": "selagi", "munas": "musyawarah nasional", "muntaber": "muntah dan berak", "musti": "mesti", "muupz": "maaf", "mw": "now watching", "n": "dan", "nanam": "menanam", "nanya": "bertanya", "napa": "kenapa", "napi": "narapidana", "napza": "narkotika, alkohol, psikotropika, dan zat adiktif ", "narkoba": "narkotika, psikotropika, dan obat terlarang", "nasgor": "nasi goreng", "nda": "tidak", "ndiri": "sendiri", "ne": "ini", "nekolin": "neokolonialisme", "nembak": "menyatakan cinta", "ngabuburit": "menunggu berbuka puasa", "ngaku": "mengaku", "ngambil": "mengambil", "nganggur": "tidak punya pekerjaan", "ngapah": "kenapa", "ngaret": "terlambat", "ngasih": "memberikan", "ngebandel": "berbuat bandel", "ngegosip": "bergosip", "ngeklaim": "mengklaim", "ngeksis": "menjadi eksis", "ngeles": "berkilah", "ngelidur": "menggigau", "ngerampok": "merampok", "ngga": "tidak", "ngibul": "berbohong", "ngiler": "mau", "ngiri": "iri", "ngisiin": "mengisikan", "ngmng": "bicara", "ngomong": "bicara", "ngubek2": "mencari-cari", "ngurus": "mengurus", "nie": "ini", "nih": "ini", "niyh": "nih", "nmr": "nomor", "nntn": "nonton", "nobar": "nonton bareng", "np": "now playing", "ntar": "nanti", "ntn": "nonton", "numpuk": "bertumpuk", "nutupin": "menutupi", "nyari": "mencari", "nyekar": "menyekar", "nyicil": "mencicil", "nyoblos": "mencoblos", "nyokap": "ibu", "ogah": "tidak mau", "ol": "online", "ongkir": "ongkos kirim", "oot": "out of topic", "org2": "orang-orang", "ortu": "orang tua", "otda": "otonomi daerah", "otw": "on the way, sedang di jalan", "pacal": "pacar", "pake": "pakai", "pala": "kepala", "pansus": "panitia khusus", "parpol": "partai politik", "pasutri": "pasangan suami istri", "pd": "pada", "pede": "percaya diri", "pelatnas": "pemusatan latihan nasional", "pemda": "pemerintah daerah", "pemkot": "pemerintah kota", "pemred": "pemimpin redaksi", "penjas": "pendidikan jasmani", "perda": "peraturan daerah", "perhatiin": "perhatikan", "pesenan": "pesanan", "pgang": "pegang", "pi": "tapi", "pilkada": "pemilihan kepala daerah", "pisan": "sangat", "pk": "penjahat kelamin", "plg": "paling", "pmrnth": "pemerintah", "polantas": "polisi lalu lintas", "ponpes": "pondok pesantren", "pp": "pulang pergi", "prg": "pergi", "prnh": "pernah", "psen": "pesan", "pst": "pasti", "pswt": "pesawat", "pw": "posisi nyaman", "qmu": "kamu", "rakor": "rapat koordinasi", "ranmor": "kendaraan bermotor", "re": "reply", "ref": "referensi", "rehab": "rehabilitasi", "rempong": "sulit", "repp": "balas", "restik": "reserse narkotika", "rhs": "rahasia", "rmh": "rumah", "ru": "baru", "ruko": "rumah toko", "rusunawa": "rumah susun sewa", "ruz": "terus", "saia": "saya", "salting": "salah tingkah", "sampe": "sampai", "samsek": "sama sekali", "sapose": "siapa", "satpam": "satuan pengamanan", "sbb": "sebagai berikut", "sbh": "sebuah", "sbnrny": "sebenarnya", "scr": "secara", "sdgkn": "sedangkan", "sdkt": "sedikit", "se7": "setuju", "sebelas dua belas": "mirip", "sembako": "sembilan bahan pokok", "sempet": "sempat", "sendratari": "seni drama tari", "sgt": "sangat", "shg": "sehingga", "siech": "sih", "sikon": "situasi dan kondisi", "sinetron": "sinema elektronik", "siramin": "siramkan", "sj": "saja", "skalian": "sekalian", "sklh": "sekolah", "skt": "sakit", "slesai": "selesai", "sll": "selalu", "slma": "selama", "slsai": "selesai", "smpt": "sempat", "smw": "semua", "sndiri": "sendiri", "soljum": "sholat jumat", "songong": "sombong", "sory": "maaf", "sosek": "sosial-ekonomi", "sotoy": "sok tahu", "spa": "siapa", "sppa": "siapa", "spt": "seperti", "srtfkt": "sertifikat", "stiap": "setiap", "stlh": "setelah", "suk": "masuk", "sumpek": "sempit", "syg": "sayang", "t4": "tempat", "tajir": "kaya", "tau": "tahu", "taw": "tahu", "td": "tadi", "tdk": "tidak", "teh": "kakak perempuan", "telat": "terlambat", "telmi": "telat berpikir", "temen": "teman", "tengil": "menyebalkan", "tepar": "terkapar", "tggu": "tunggu", "tgu": "tunggu", "thankz": "terima kasih", "thn": "tahun", "tilang": "bukti pelanggaran", "tipiwan": "TvOne", "tks": "terima kasih", "tlp": "telepon", "tls": "tulis", "tmbah": "tambah", "tmen2": "teman-teman", "tmpah": "tumpah", "tmpt": "tempat", "tngu": "tunggu", "tnyta": "ternyata", "tokai": "tai", "toserba": "toko serba ada", "tpi": "tapi", "trdhulu": "terdahulu", "trima": "terima kasih", "trm": "terima", "trs": "terus", "trutama": "terutama", "ts": "penulis", "tst": "tahu sama tahu", "ttg": "tentang", "tuch": "tuh", "tuir": "tua", "tw": "tahu", "u": "kamu", "ud": "sudah", "udah": "sudah", "ujg": "ujung", "ul": "ulangan", "unyu": "lucu", "uplot": "unggah", "urang": "saya", "usah": "perlu", "utk": "untuk", "valas": "valuta asing", "w/": "dengan", "wadir": "wakil direktur", "wamil": "wajib militer", "warkop": "warung kopi", "warteg": "warung tegal", "wat": "buat", "wkt": "waktu", "wtf": "what the fuck", "xixixi": "tertawa", "ya": "iya", "yap": "iya", "yaudah": "ya sudah", "yawdah": "ya sudah", "yg": "yang", "yl": "yang lain", "yo": "iya", "yowes": "ya sudah", "yup": "iya", "7an": "tujuan", "ababil": "abg labil", "acc": "accord", "adlah": "adalah", "adoh": "aduh", "aha": "tertawa", "aing": "saya", "aja": "saja", "ajj": "saja", "aka": "dikenal juga sebagai", "akko": "aku", "akku": "aku", "akyu": "aku", "aljasa": "asal jadi saja", "ama": "sama", "ambl": "ambil", "anjir": "anjing", "ank": "anak", "ap": "apa", "apaan": "apa", "ape": "apa", "aplot": "unggah", "apva": "apa", "aqu": "aku", "asap": "sesegera mungkin", "aseek": "asyik", "asek": "asyik", "aseknya": "asyiknya", "asoy": "asyik", "astrojim": "astagfirullahaladzim", "ath": "kalau begitu", "atuh": "kalau begitu", "ava": "avatar", "aws": "awas", "ayang": "sayang", "ayok": "ayo", "bacot": "banyak bicara", "bales": "balas", "bangdes": "pembangunan desa", "bangkotan": "tua", "banpres": "bantuan presiden", "bansarkas": "bantuan sarana kesehatan", "bazis": "badan amal, zakat, infak, dan sedekah", "bcoz": "karena", "beb": "sayang", "bejibun": "banyak", "belom": "belum", "bener": "benar", "ber2": "berdua", "berdikari": "berdiri di atas kaki sendiri", "bet": "banget", "beti": "beda tipis", "beut": "banget", "bgd": "banget", "bgs": "bagus", "bhubu": "tidur", "bimbuluh": "bimbingan dan penyuluhan", "bisi": "kalau-kalau", "bkn": "bukan", "bl": "beli", "blg": "bilang", "blm": "belum", "bls": "balas", "bnchi": "benci", "bngung": "bingung", "bnyk": "banyak", "bohay": "badan aduhai", "bokep": "porno", "bokin": "pacar", "bole": "boleh", "bolot": "bodoh", "bonyok": "ayah ibu", "bpk": "bapak", "brb": "segera kembali", "brngkt": "berangkat", "brp": "berapa", "brur": "saudara laki-laki", "bsa": "bisa", "bsk": "besok", "bu_bu": "tidur", "bubarin": "bubarkan", "buber": "buka bersama", "bujubune": "luar biasa", "buser": "buru sergap", "bwhn": "bawahan", "byar": "bayar", "byr": "bayar", "c8": "chat", "cabut": "pergi", "caem": "cakep", "cama-cama": "sama-sama", "cangcut": "celana dalam", "cape": "capek", "caur": "jelek", "cekak": "tidak ada uang", "cekidot": "coba lihat", "cemplungin": "cemplungkan", "ceper": "pendek", "ceu": "kakak perempuan", "cewe": "cewek", "cibuk": "sibuk", "cin": "cinta", "ciye": "cie", "ckck": "ck", "clbk": "cinta lama bersemi kembali", "cmpr": "campur", "cnenk": "senang", "congor": "mulut", "cow": "cowok", "coz": "karena", "cpa": "siapa", "gokil": "gila", "gombal": "suka merayu", "gpl": "tidak pakai lama", "gpp": "tidak apa-apa", "gretong": "gratis", "gt": "begitu", "gtw": "tidak tahu", "gue": "saya", "guys": "teman-teman", "gws": "cepat sembuh", "haghaghag": "tertawa", "hakhak": "tertawa", "handak": "bahan peledak", "hansip": "pertahanan sipil", "hellow": "halo", "helow": "halo", "hi": "hai", "hlng": "hilang", "hnya": "hanya", "houm": "rumah", "hrs": "harus", "hubad": "hubungan angkatan darat", "hubla": "perhubungan laut", "huft": "mengeluh", "humas": "hubungan masyarakat", "idk": "saya tidak tahu", "ilfeel": "tidak suka", "imba": "jago sekali", "imoet": "imut", "info": "informasi", "itung": "hitung", "isengin": "bercanda", "iyala": "iya lah", "iyo": "iya", "jablay": "jarang dibelai", "jadul": "jaman dulu", "jancuk": "anjing", "jd": "jadi", "jdikan": "jadikan", "jg": "juga", "jgn": "jangan", "jijay": "jijik", "jkt": "jakarta", "jnj": "janji", "jth": "jatuh", "jurdil": "jujur adil", "jwb": "jawab", "ka": "kakak", "kabag": "kepala bagian", "kacian": "kasihan", "kadit": "kepala direktorat", "kaga": "tidak", "kaka": "kakak", "kamtib": "keamanan dan ketertiban", "kamuh": "kamu", "kamyu": "kamu", "kapt": "kapten", "kasat": "kepala satuan", "kasubbid": "kepala subbidang", "kau": "kamu", "kbar": "kabar", "kcian": "kasihan", "keburu": "terlanjur", "kedubes": "kedutaan besar", "kek": "seperti", "keknya": "kayaknya", "keliatan": "kelihatan", "keneh": "masih", "kepikiran": "terpikirkan", "kepo": "mau tahu urusan orang", "kere": "tidak punya uang", "kesian": "kasihan", "ketauan": "ketahuan", "keukeuh": "keras kepala", "khan": "kan", "kibus": "kaki busuk", "kk": "kakak", "klian": "kalian", "klo": "kalau", "kluarga": "keluarga", "klwrga": "keluarga", "kmari": "kemari", "kmpus": "kampus", "kn": "kan", "knl": "kenal", "knpa": "kenapa", "kog": "kok", "kompi": "komputer", "komtiong": "komunis Tiongkok", "konjen": "konsulat jenderal", "koq": "kok", "kpd": "kepada", 
        "kptsan":"keputusan", "krik": "garing", "krn": "karena", "ktauan": "ketahuan", "ktny": "katanya", "kudu": "harus", "kuq": "kok", "ky": "seperti", "kykny": "kayanya", "laka": "kecelakaan", 
        "lambreta": "lambat", "lansia": "lanjut usia", "lapas": "lembaga pemasyarakatan", "lbur": "libur", "lekong": "laki-laki", "lg": "lagi", "lgkp": "lengkap", "lht": "lihat", 
        "linmas": "perlindungan masyarakat", "lmyan": "lumayan", "lngkp": "lengkap", "loch": "loh", "lol": "tertawa", "lom": "belum", "loupz": "cinta", "lowh": "kamu", "lu": "kamu", 
        "luchu": "lucu", "luff": "cinta", "luph": "cinta", "lw": "kamu", "lwt": "lewat", "maaciw": "terima kasih", "mabes": "markas besar", "macem-macem": "macam-macam", "madesu": "masa depan suram", 
        "maen": "main", "mahatma": "maju sehat bersama", "mak": "ibu", "makasih": "terima kasih", "malah": "bahkan", "malu2in": "memalukan", "mamz": "makan", "manies": "manis", "mantep": "mantap", 
        "markus": "makelar kasus", "mba": "mbak", "mending": "lebih baik", "mgkn": "mungkin", "mhn": "mohon", "miker": "minuman keras", "milis": "mailing list", "mksd": "maksud", "mls": "malas", 
        "mnt": "minta", "moge": "motor gede", "mokat": "mati", "mosok": "masa", "msh": "masih", "mskpn": "meskipun", "msng2": "masing-masing", "muahal": "mahal", "muker": "musyawarah kerja", 
        "mumet": "pusing", "muna": "munafik", "munaslub": "musyawarah nasional luar biasa", "musda": "musyawarah daerah", "muup": "maaf", "muuv": "maaf", "nal": "kenal", "nangis": "menangis", 
        "naon": "apa", "napol": "narapidana politik", "naq": "anak", "narsis": "bangga pada diri sendiri", "nax": "anak", "ndak": "tidak", "ndut": "gendut", "nekolim": "neokolonialisme", 
        "nelfon": "menelepon", "ngabis2in": "menghabiskan", "ngakak": "tertawa", "ngambek": "marah", "ngampus": "pergi ke kampus", "ngantri": "mengantri", "ngapain": "sedang apa", 
        "ngaruh": "berpengaruh", "ngawur": "berbicara sembarangan", "ngeceng": "kumpul bareng-bareng", "ngeh": "sadar", "ngekos": "tinggal di kos", "ngelamar": "melamar", "ngeliat": "melihat", 
        "ngemeng": "bicara terus-terusan", "ngerti": "mengerti", "nggak": "tidak", "ngikut": "ikut", "nginep": "menginap", "ngisi": "mengisi", "ngmg": "bicara", "ngocol": "lucu", 
        "ngomongin": "membicarakan", "ngumpul": "berkumpul", "ni": "ini", "nyasar": "tersesat", "nyariin": "mencari", "nyiapin": "mempersiapkan", "nyiram": "menyiram", "nyok": "ayo", "o/": "oleh", 
        "ok": "ok", "priksa": "periksa", "pro": "profesional", "psn": "pesan", "psti": "pasti", "puanas": "panas", "qmo": "kamu", "qt": "kita", "rame": "ramai", "raskin": "rakyat miskin", 
        "red": "redaksi", "reg": "register", "rejeki": "rezeki", "renstra": "rencana strategis", "reskrim": "reserse kriminal", "sni": "sini", "somse": "sombong sekali", "sorry": "maaf", 
        "sosbud": "sosial-budaya", "sospol": "sosial-politik", "sowry": "maaf", "spd": "sepeda", "sprti": "seperti", "spy": "supaya", "stelah": "setelah", "subbag": "subbagian", 
        "sumbangin": "sumbangkan", "sy": "saya", "syp": "siapa", "tabanas": "tabungan pembangunan nasional", "tar": "nanti", "taun": "tahun", "tawh": "tahu", "tdi": "tadi", "te2p": "tetap", 
        "tekor": "rugi", "telkom": "telekomunikasi", "telp": "telepon", "temen2": "teman-teman", "tengok": "menjenguk", "terbitin": "terbitkan", "tgl": "tanggal", "thanks": "terima kasih", 
        "thd": "terhadap", "thx": "terima kasih", "tipi": "TV", "tkg": "tukang", "tll": "terlalu", "tlpn": "telepon", "tman": "teman", "tmbh": "tambah", "tmn2": "teman-teman", "tmph": "tumpah", 
        "tnda": "tanda", "tnh": "tanah", "togel": "toto gelap", "tp": "tapi", "tq": "terima kasih", "trgntg": "tergantung", "trims": "terima kasih", "cb": "coba", "y": "ya", "munfik": "munafik", 
        "reklamuk": "reklamasi", "sma": "sama", "tren": "trend", "ngehe": "kesal", "mz": "mas", "analisise": "analisis", "sadaar": "sadar", "sept": "september", "nmenarik": "menarik", 
        "zonk": "bodoh", "rights": "benar", "simiskin": "miskin", "ngumpet": "sembunyi", "hardcore": "keras", "akhirx": "akhirnya", "solve": "solusi", "watuk": "batuk", "ngebully": "intimidasi", 
        "masy": "masyarakat", "still": "masih", "tauk": "tahu", "mbual": "bual", "tioghoa": "tionghoa", "ngentotin": "senggama", "kentot": "senggama", "faktakta": "fakta", "sohib": "teman", 
        "rubahnn": "rubah", "trlalu": "terlalu", "nyela": "cela", "heters": "pembenci", "nyembah": "sembah", "most": "paling", "ikon": "lambang", "light": "terang", "pndukung": "pendukung", 
        "setting": "atur", "seting": "akting", "next": "lanjut", "waspadalah": "waspada", "gantengsaya": "ganteng", "parte": "partai", "nyerang": "serang", "nipu": "tipu", "ktipu": "tipu", 
        "jentelmen": "berani", "buangbuang": "buang", "tsangka": "tersangka", "kurng": "kurang", "ista": "nista", "less": "kurang", "koar": "teriak", "paranoid": "takut", "problem": "masalah", 
        "tahi": "kotoran", "tirani": "tiran", "tilep": "tilap", "happy": "bahagia", "tak": "tidak", "penertiban": "tertib", "uasai": "kuasa", "mnolak": "tolak", "trending": "trend", 
        "taik": "tahi", "wkwkkw": "tertawa", "ahokncc": "ahok", "istaa": "nista", "benarjujur": "jujur", "mgkin": "mungkin", 'ga':'tidak', 'cwe':'perempuan', 'dl':'dulu', 'rumayan':'lumayan',
        'ny':'nya', 'htm':'harga tiket masuk', 'cm':'cuma', 'slalu':'selalu', 'tingi':'tinggi','neng':'senang'}
processed_comments = []

for sentence in data['text']:
    # Remove all the special characters
    processed_comment = re.sub(r'\W', ' ', str(sentence))
    
    # Converting to Lowercase
    processed_comment = processed_comment.lower()
    
    #Remove number 
    processed_comment = re.sub(r'\d+', ' ', processed_comment)

    # remove all single characters
    processed_comment = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_comment)
    
    #remove duplicate character
    pattern=reg.compile(r"(.)\1{1,}",reg.DOTALL)
    processed_comment=pattern.sub(r"\1",processed_comment)


    #Corrected Slang words
    words = processed_comment.split()
    rfrm=[slangs[word] if word in slangs else word for word in words]
    processed_comment= " ".join(rfrm)

    #remove stopword
    factory = StopWordRemoverFactory()
    more_stopword = ['tak', 'jd', 'per', 'nya', 'terjemah', 'diterjemahkan', 'oleh', 'gogle', 'google' ,'nan', 'baik', 'sangat', 'batas', 'coba',
                     'ada','bersih', 'salur', 'baru', 'purwokerto', 'batas', 'hotel', 'coba', 'putus', 'ada', 
                     'com', 'kamu', 'http', 'https', 'htps', 'htp', 'gak', 'jadi', 'lebih', 'kalau', 'banyak', 'jangan', 'iya'] #menambahkan stopword
    stopwords = factory.get_stop_words() + more_stopword
    temp = [t for t in re.findall(r'\b[a-z]+-?[a-z]+\b',processed_comment) if t not in stopwords]
    processed_comment = ' '.join(temp)
    
    #stemming
    stemmer = StemmerFactory().create_stemmer()
    processed_comment = stemmer.stem(processed_comment)
    
    # Substituting multiple spaces with single space
    processed_comment = re.sub(r'\s+', ' ', processed_comment, flags=re.I)

    processed_comments.append(processed_comment)

In [14]:
processed_comments

['apa harap politis gaya lacur pramagtis jelas warna politik punya prinsip politik sama milik integritas cukup yakin politis model begini suka rakyat',
 'syahganda gugat syahganda mg benar mg lacur intlektual syahganda buka bukan propaganda hasil survei deny depan bawaslu politik rmol co read syahganda buka bukan propaganda hasil survei deny depan bawaslu',
 'pdemokrat org punya akal sehat maju bangsa beberapa orang begundal serta lacur politik grasak grusuk salah satu heran twiter ferdinandhaean status',
 'cypridophobia fobia takut lacur tular sakit kelamin',
 'ramai kerja waktu malam biasa lacur siapa idea suh kerja malam tu takfahamtakapa',
 'lacur intlektual kpucurangrakyatmelawan kpungejarquickcounthtps twiter suwandaben status',
 'lembaga lacur lsi denyja germo rupa kejar target atas propinsi opini publik simultan dg operasi ampo suara mana hina betul hidup merekahtps twiter status',
 'bangun gada orang sukses kasur lacur selfreminder',
 'abai semua lacur agama hidup nkri hidup p

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
vektor = TfidfVectorizer(max_features=400)

In [16]:
vektor

TfidfVectorizer(max_features=400)

In [19]:
df = pd.DataFrame(processed_comments,
                 columns=['CleanText'])
df

,CleanText
0,apa harap politis gaya lacur pramagtis jelas w...
1,syahganda gugat syahganda mg benar mg lacur in...
2,pdemokrat org punya akal sehat maju bangsa beb...
3,cypridophobia fobia takut lacur tular sakit ke...
4,ramai kerja waktu malam biasa lacur siapa idea...
...,...
395,leta palak sepatu mau kotor kq repot banget la...
396,media lacur lahir rahim jujur twiter status
397,lacur ko dijawa nak
398,kok lacur


In [20]:
#menghitung tf-idf dengan TfidfTransformer
vektor_dt=vektor.fit_transform(df['CleanText'].values.astype('U'))
print (vektor_dt)
print (vektor_dt.shape)

  (0, 322)	0.23428545062335412
  (0, 356)	0.21403668591215794
  (0, 41)	0.2800446418971443
  (0, 74)	0.26890094253306285
  (0, 257)	0.2800446418971443
  (0, 330)	0.18468285585384706
  (0, 320)	0.19966995871364537
  (0, 309)	0.3252358111875643
  (0, 397)	0.26890094253306285
  (0, 157)	0.24542914998743556
  (0, 209)	0.06734427420447217
  (0, 310)	0.5041953082487916
  (0, 122)	0.2520976541243958
  (0, 15)	0.18852625934956393
  (1, 38)	0.29012595329817603
  (1, 85)	0.24842815157491535
  (1, 84)	0.29012595329817603
  (1, 359)	0.29012595329817603
  (1, 123)	0.2308752203244861
  (1, 316)	0.29012595329817603
  (1, 68)	0.19676358727159762
  (1, 67)	0.24842815157491535
  (1, 140)	0.13249342668868727
  (1, 46)	0.09964393397575408
  (1, 256)	0.2759683464195877
  :	:
  (395, 249)	0.4234008144551638
  (395, 25)	0.19078642077414176
  (395, 200)	0.22409274937034854
  (395, 346)	0.17676835669132596
  (395, 209)	0.05808931120962021
  (396, 164)	0.49349564661662076
  (396, 249)	0.43249610580338044
  (396

In [21]:
idf=vektor.idf_

In [24]:
dd = dict(zip(vektor.get_feature_names(), idf))
l=sorted(dd, key=(dd).get)
print(l[0],l[-1])
print(dd['jadi'])
print(dd['lacur'])

lacur wajah
3.9494389895831463
1.3485145296633316


# Topik Modeling menggunakan LSA

In [25]:
from sklearn.decomposition import TruncatedSVD
lsa_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=10, random_state=42)
lsa_top=lsa_model.fit_transform(vektor_dt)

In [27]:
print(lsa_top.shape)

(400, 10)


In [28]:
print(lsa_top)

[[ 0.28166442 -0.03981739  0.23811337 ...  0.05576784  0.24788922
  -0.02100457]
 [ 0.09469228 -0.011208    0.04902164 ...  0.03347066  0.0167774
   0.05787912]
 [ 0.3752088   0.05602625  0.0545989  ... -0.15848653  0.27653385
  -0.12137331]
 ...
 [ 0.161605   -0.02007222 -0.07123389 ...  0.05001458 -0.13042724
  -0.095935  ]
 [ 0.23885015  0.00163864 -0.0750316  ...  0.07776335  0.02039774
  -0.03702511]
 [ 0.21958257  0.06273732 -0.12061758 ... -0.0006633  -0.05585936
   0.31365805]]


In [30]:
# Memunculkan nilai lsa setiap topik
r = lsa_top[0]
print("Topik-topik:")
for i,topic in enumerate(r):
 print("Topic ",i," : ",topic*100)


Topik-topik:
Topic  0  :  28.166441646803825
Topic  1  :  -3.9817389462145156
Topic  2  :  23.811336509531852
Topic  3  :  11.671638896061657
Topic  4  :  -0.020884031554074367
Topic  5  :  5.1392154872932725
Topic  6  :  -18.897150449518314
Topic  7  :  5.576784367491536
Topic  8  :  24.788921807971
Topic  9  :  -2.1004565625625102


In [31]:
# Memunculkan jumlah kata-kata dalam setiap topik
print(lsa_model.components_.shape)
print(lsa_model.components_)

(10, 400)
[[ 1.82868575e-02  5.59284184e-02  1.84589636e-02 ...  1.26644319e-02
   1.86308285e-02  6.19546311e-03]
 [ 2.92715066e-04  2.28120788e-02  5.30378502e-02 ...  6.44408960e-03
   4.60893881e-02  1.28764017e-03]
 [-1.67669257e-02 -1.76021253e-02 -1.35209829e-02 ...  1.48647335e-02
   1.02649276e-02  6.44392738e-04]
 ...
 [ 1.19699010e-02  7.98053718e-02 -4.68522318e-02 ...  1.43449637e-02
  -1.71229585e-02  6.28536205e-03]
 [-2.69659450e-02  5.70728907e-02  2.85291227e-02 ...  3.60458703e-02
  -1.07870493e-02 -6.63284353e-03]
 [ 9.22889337e-02  3.31840324e-01  1.00588083e-01 ... -3.80644091e-03
   7.74497579e-03 -1.24961106e-03]]


In [32]:
# Word/ kata paling penting dalam setiap topik
vocab = vektor.get_feature_names()
for i, comp in enumerate(lsa_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+" : ")
    for g in sorted_words:
        print(g[0],end=", ")
    print("\n")

Topic 0 : 
lacur, politik, orang, twiter, status, aku, ras, jadi, apa, semua, 

Topic 1 : 
ras, twiter, orang, status, suka, mesum, anjing, sama, cina, mau, 

Topic 2 : 
politik, ras, negri, org, jelas, cina, satu, rakyat, lah, politis, 

Topic 3 : 
mesum, politik, otak, twiter, status, punya, maf, pic, kan, lihat, 

Topic 4 : 
twiter, status, intlektual, leceh, pic, verbal, politik, bukan, tutup, perempuan, 

Topic 5 : 
aku, izin, tuhan, jadi, politik, binatang, paksa, hidup, baca, ras, 

Topic 6 : 
lacur, ras, mesum, intelektual, twiter, maf, pic, bekas, negara, bawa, 

Topic 7 : 
leceh, verbal, anjing, seksual, nama, baca, sama, bodoh, apa, biar, 

Topic 8 : 
punya, anak, baca, bangsa, politis, jadi, org, ibu, rakyat, agama, 

Topic 9 : 
agama, tutup, tusuk, semua, prabowo, dasar, hidup, mau, negara, sandiuno, 



# Topik Modeling menggunakan LDA

In [38]:
import nltk

In [39]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [40]:
from nltk.tokenize import word_tokenize

Docs = str(processed_comments)
Sentences = nltk.sent_tokenize(Docs)
print ("Kalimat Awal: ", Sentences)
hasil_tokenizing = nltk.word_tokenize(Docs)
print("Setelah Tokenizing: ", hasil_tokenizing)

Kalimat Awal:  ["['apa harap politis gaya lacur pramagtis jelas warna politik punya prinsip politik sama milik integritas cukup yakin politis model begini suka rakyat', 'syahganda gugat syahganda mg benar mg lacur intlektual syahganda buka bukan propaganda hasil survei deny depan bawaslu politik rmol co read syahganda buka bukan propaganda hasil survei deny depan bawaslu', 'pdemokrat org punya akal sehat maju bangsa beberapa orang begundal serta lacur politik grasak grusuk salah satu heran twiter ferdinandhaean status', 'cypridophobia fobia takut lacur tular sakit kelamin', 'ramai kerja waktu malam biasa lacur siapa idea suh kerja malam tu takfahamtakapa', 'lacur intlektual kpucurangrakyatmelawan kpungejarquickcounthtps twiter suwandaben status', 'lembaga lacur lsi denyja germo rupa kejar target atas propinsi opini publik simultan dg operasi ampo suara mana hina betul hidup merekahtps twiter status', 'bangun gada orang sukses kasur lacur selfreminder', 'abai semua lacur agama hidup nkr

In [41]:
#Ekstraksi fitur / representasi dokumen menggunakan TF
from sklearn.feature_extraction.text import CountVectorizer
tf_vectorizer = CountVectorizer(max_df=1.0, min_df=1)
tf = tf_vectorizer.fit_transform(hasil_tokenizing)
#hasil representasi
tf_terms = tf_vectorizer.get_feature_names()
print(tf_vectorizer.get_feature_names())
matrix = tf.toarray()
print(matrix)

['abai', 'abal', 'abdu', 'abg', 'abng', 'abu', 'abud', 'ada', 'adab', 'adik', 'adil', 'adrenalin', 'aduh', 'ae', 'agama', 'agenda', 'agt', 'ah', 'ahahaha', 'ahoax', 'ahok', 'air', 'ajah', 'ajak', 'ajakin', 'ajar', 'akal', 'akbar', 'akhir', 'akhirat', 'akomentar', 'aku', 'akubacadotcom', 'akun', 'akwkwk', 'ala', 'alah', 'alam', 'alas', 'alhamdulilah', 'ali', 'alias', 'alim', 'alir', 'alohu', 'alus', 'aman', 'ambil', 'amca', 'amerika', 'amien', 'amin', 'ampas', 'amplop', 'ampo', 'ampum', 'ampun', 'an', 'anak', 'analogi', 'anarkis', 'anatomi', 'ancam', 'and', 'anda', 'andai', 'andik', 'andre', 'ane', 'aneh', 'angap', 'anget', 'angin', 'angkat', 'angkatanya', 'angota', 'aniesbaswedan', 'anime', 'anjeng', 'anjing', 'anti', 'anuarmusa', 'anus', 'anying', 'apa', 'apan', 'aparat', 'apresiasi', 'apung', 'arab', 'arah', 'are', 'arek', 'argentina', 'argumen', 'arow', 'arti', 'arus', 'as', 'asa', 'asal', 'asli', 'asyik', 'atas', 'atlet', 'atu', 'atur', 'audience', 'australia', 'av', 'avatar', 'awa

In [42]:
# panggil class LDA
from sklearn.decomposition import LatentDirichletAllocation as LDA

n_topics = 10 #untuk mendapatkan jumlah topik terbaik perlu trial
lda = LDA(n_components=n_topics, learning_method='batch', random_state=0).fit(tf)
lda

LatentDirichletAllocation(random_state=0)

In [43]:
#training LDA
vsm_topics = lda.transform(tf)
#tampilkan hasil
print(vsm_topics.shape)
vsm_topics

(5899, 10)


array([[0.1       , 0.1       , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ],
       [0.05      , 0.05      , 0.05      , ..., 0.05      , 0.55      ,
        0.05      ],
       [0.05      , 0.05      , 0.05      , ..., 0.05      , 0.05      ,
        0.05      ],
       ...,
       [0.05      , 0.05      , 0.54999999, ..., 0.05      , 0.05      ,
        0.05      ],
       [0.1       , 0.1       , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ],
       [0.1       , 0.1       , 0.1       , ..., 0.1       , 0.1       ,
        0.1       ]])

In [44]:
#Tampilkan nilai-nilai setiap fitur
print(lda.components_)

[[2.09999999 0.1        0.1        ... 0.1        0.1        0.1       ]
 [0.1        0.1        0.1        ... 0.1        0.1        0.1       ]
 [0.1        0.1        0.1        ... 3.09999999 1.09999998 0.1       ]
 ...
 [0.1        0.1        0.1        ... 0.1        0.1        3.09999999]
 [0.1        1.09999998 0.1        ... 0.1        0.1        0.1       ]
 [0.1        0.1        6.09999999 ... 0.1        0.1        0.1       ]]


In [45]:
#hasil label topic model untuk setiap dokumen
import numpy as np
topics = np.argmax(vsm_topics, axis=1)
topics

array([0, 8, 5, ..., 2, 0, 0], dtype=int64)

In [46]:
#mencetak word fitur dengan nilai tertinggi pada setiap topik
n_top_words = 4 # jumlah fitur tertinggi yang kita tentukan
topic_words = {}
for topic, comp in enumerate(lda.components_): 
 word_idx = np.argsort(comp)[::-1][:n_top_words]
 # store the words most relevant to the topic
 topic_words[topic] = [tf_vectorizer.get_feature_names()[i]+' '+str(comp[i]) for i in word_idx]

In [47]:
for topic, words in topic_words.items():
 print('Topic: %d' % topic)
 print(' %s' % ', '.join(words))

Topic: 0
 sama 27.099999988365532, bukan 23.099999988335306, agama 21.099999988315798, ahok 16.099999988245315
Topic: 1
 jadi 23.09999998757867, org 16.099999987482867, perempuan 15.09999998746173, baca 13.099999987409504
Topic: 2
 aku 40.099999987532414, verbal 24.099999987448463, mau 23.099999987439308, kata 17.09999998736122
Topic: 3
 anjing 21.09999998767758, buat 15.09999998758232, lah 15.09999998758232, jual 12.099999987498107
Topic: 4
 lacur 316.09999998432943, politik 45.099999984209994, dasar 15.09999998392442, lihat 14.099999983893374
Topic: 5
 leceh 26.09999998788197, hina 15.099999987743114, negeri 15.09999998774311, tu 14.09999998771944
Topic: 6
 benar 21.099999990469765, biasa 12.099999990330964, kok 11.099999990301152, pic 10.099999990265204
Topic: 7
 orang 63.09999998716443, status 42.099999987123525, bilang 20.099999986986976, punya 19.09999998697313
Topic: 8
 ras 68.0999999869539, twiter 56.09999998693752, mesum 28.09999998684336, apa 24.09999998681179
Topic: 9
 laku 

# Kesimpulan

Hasil kata teratas dari masing-masing Topic Modelling menggunakan LSA:
1. Topik 1: Lacur
2. Topik 2: Ras
3. Topik 3: Politik
4. Topik 4: Mesum
5. Topik 5: Twitter
6. Topik 6: Aku
7. Topik 7: Lacur
8. TOpik 8: Receh
9. TOpik 9: Punya
10. Topik 10: Agama

Hasil kata teratas dari masing-masing Topic Modelling menggunakan LDA:
1. Topik 1: Sama
2. Topik 2: Jadi
3. Topik 3: Aku
4. Topik 4: Anjing
5. Topik 5: Lacur
6. Topik 6: Receh
7. Topik 7: Benar
8. TOpik 8: Orang
9. TOpik 9: Ras
10. Topik 10: Laku

Dari hasil di atas, terdapat perbedaan perolehan kata dari masing-masing topik. Hal ini dikarenakan proses yang dijalankan masing-masing metode. Dari hasil di atas, metode yang baik yaitu menggunakan metode LSA.